### A/B Test – Recommendation System Algorithm Update  
#### Data Cleaning

This notebook documents the data preparation stage of a two-sided A/B test evaluating an updated recommendation algorithm for a streaming platform. The dataset contains simulated session-level user interaction data, where users are randomly assigned to either a control group (existing algorithm) or a treatment group (updated algorithm).

Column description:

- `user_id`: Unique identifier for each user
- `session_id`: Unique identifier for each session
- `group`: Experimental assignment (control or treatment)
- `session_start`, `session_end`: Session timestamps
- `impressions`: Number of recommended items shown
- `clicks`: Number of recommended items clicked
- `content_started`: Number of recommended content items initiated
- `content_completed`: Number of recommended items watched to completion
- `total_watch_time_minutes`: Total watch time within the session (any content source)
- `device_type`, `os`, `app_version`: Technical metadata
- `language_preference`, `referral_source`, `subscription_type`: User metadata
- `is_premium_user`: Indicator of subscription status

The purpose of this phase is to ensure the dataset is accurate, consistent, and analysis-ready before conducting exploratory analysis and statistical testing.

#### Objectives of this phase:
- Inspect dataset structure, schema, and overall integrity
- Identify and resolve data quality issues (e.g., missing values, duplicates, incorrect data types)
- Validate logical consistency across related variables
- Prepare a cleaned dataset to be used in subsequent exploratory and statistical analysis

The cleaned dataset produced in this notebook will serve as the foundation for metric calculation, hypothesis testing, and business interpretation in later stages of the project.

#### 1. Imports and Data Loading

This section initializes the core libraries required for data manipulation and loads the raw session-level dataset into a pandas DataFrame for inspection and cleaning.

The dataset is sourced from the data/raw directory to preserve the integrity of the original file.

In [1]:
# Import core libraries for data manipulation 
import pandas as pd
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv("../data/raw/streaming_ab_test_sessions.csv")

#### 2. Initial Data Inspection

This section provides an initial overview of the dataset to assess structure, completeness, and potential data quality issues.

This includes:
- Previewing sample records
- Checking dataset dimensions
- Inspecting column types and missing values
- Identifying duplicated rows

In [3]:
# Preview the first ten rows
df.head(10)

,user_id,group,session_id,session_start,session_end,impressions,clicks,content_started,content_completed,total_watch_time_minutes,device_type,os,app_version,language_preference,referral_source,subscription_type,is_premium_user
0,8116,control,48680,2026-01-11 12:07:36.131448076,2026-01-11 12:52:36.131448076,21,6,4,1,22.08,tablet,Android,5.0.0,Spanish,push_notification,Basic,False
1,12109,control,72536,2026-01-14 10:02:48.687211687,2026-01-14 11:11:48.687211687,24,8,8,2,37.17,smart_tv,Andriod,version1',tr-TR,push_notification,Basic,0
2,15861,control,94927,2026-01-14 10:28:05.164624346,2026-01-14 11:59:05.164624346,21,3,3,1,34.53,desktop,iOS,6.0.0,dutch,ad_campaign,Premium,False
3,21199,TREATMENT,127477,2026-01-03 20:33:22.917005566,2026-01-03 22:06:22.917005566,24,6,6,5,66.34,desktop,Android,6.1.1,DE,email,Premium,True
4,2324,control,13847,2026-01-18 05:00:07.290494422,2026-01-18 06:52:07.290494422,15,3,2,1,79.78,mobile,Windows,5.2.1,spanish,push_notification,premium,0
5,2843,treatment,16961,2026-01-20 18:51:25.266066025,2026-01-20 19:13:25.266066025,13,5,5,2,8.72,tablet,Android,5.2.0,English,email,Premium,True
6,17723,control,106413,2026-01-11 04:33:20.555318364,2026-01-11 05:58:20.555318364,14,2,2,1,77.71,moblie,mac,NaN,Spanish,email,Basic,No
7,13518,treatment,81025,2026-01-21 02:49:21.498920094,2026-01-21 04:29:21.498920094,23,5,5,4,-70.93,tablet,Android,5.4.0,English,push_notification,Family,true
8,2557,treatment,15269,2026-01-06 10:49:59.014963547,2026-01-06 12:21:59.014963547,14,3,3,1,78.27,desktop,iOS,5.7.0,DE,email,Family,1
9,4831,treatment,28899,2026-01-08 00:06:51.703422033,2026-01-08 01:05:51.703422033,26,2,1,0,13.98,mobile,Windows,5.5.1,dutch,push_notification,premium,1


In [4]:
# Display dataset dimensions
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Number of rows: 150079
Number of columns: 17


In [5]:
# Display column names, data types, and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150079 entries, 0 to 150078
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   user_id                   150079 non-null  int64  
 1   group                     150079 non-null  object 
 2   session_id                150079 non-null  int64  
 3   session_start             150079 non-null  object 
 4   session_end               150079 non-null  object 
 5   impressions               150079 non-null  int64  
 6   clicks                    150079 non-null  int64  
 7   content_started           150079 non-null  int64  
 8   content_completed         150079 non-null  int64  
 9   total_watch_time_minutes  150079 non-null  float64
 10  device_type               147080 non-null  object 
 11  os                        150079 non-null  object 
 12  app_version               142806 non-null  object 
 13  language_preference       141968 non-null  o

In [6]:
# Show descriptive statistics for all columns
df.describe(include='all')

,user_id,group,session_id,session_start,session_end,impressions,clicks,content_started,content_completed,total_watch_time_minutes,device_type,os,app_version,language_preference,referral_source,subscription_type,is_premium_user
count,150079.000000,150079,150079.000000,150079,150079,150079.000000,150079.000000,150079.000000,150079.000000,150079.000000,147080,150079,142806,141968,145587,145570,150079
unique,NaN,10,NaN,147414,147414,NaN,NaN,NaN,NaN,NaN,6,6,29,18,4,5,7
top,NaN,control,NaN,2026-01-10 08:01:16.449410743,2026-01-10 09:47:16.449410743,NaN,NaN,NaN,NaN,NaN,tablet,Android,6.2.0,DE,push_notification,Basic,False
freq,NaN,58280,NaN,3,3,NaN,NaN,NaN,NaN,NaN,36899,33998,5082,14291,36938,29419,37594
mean,12284.415288,NaN,73712.376169,NaN,NaN,19.899580,3.102479,2.793056,1.662558,39.042225,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,7071.234363,NaN,42549.111745,NaN,NaN,5.161377,1.800642,1.702991,1.303092,24.519867,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,1.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,-107.060000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,6159.000000,NaN,36876.500000,NaN,NaN,17.000000,2.000000,2.000000,1.000000,19.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,12305.000000,NaN,73730.000000,NaN,NaN,20.000000,3.000000,3.000000,1.000000,35.270000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,18394.000000,NaN,110560.500000,NaN,NaN,23.000000,4.000000,4.000000,2.000000,55.730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Count missing values per column
df.isnull().sum()

user_id                        0
group                          0
session_id                     0
session_start                  0
session_end                    0
impressions                    0
clicks                         0
content_started                0
content_completed              0
total_watch_time_minutes       0
device_type                 2999
os                             0
app_version                 7273
language_preference         8111
referral_source             4492
subscription_type           4509
is_premium_user                0
dtype: int64

In [8]:
# Check for fully duplicated rows
print(f"Number of fully duplicated rows:{df.duplicated().sum()}")

Number of fully duplicated rows:1223


##### **Key findings from Initial Data Inspection:**

- The dataset contains **150,079 rows** and **17 columns**, with each row representing a user session.
- Core experimental columns (`user_id`, `group`,  `session_id`, `session_start`, `session_end`, `impressions`, and `clicks`, `content_started`, `content_completed`, `total_watch_time_minutes`) are complete with no missing values.
- Missing values are limited to secondary metadata columns:
  - `device_type`: 2,999 missing values
  - `app_version`: 7,273 missing values
  - `language_preference`: 8,111 missing values
  - `referral_source`: 4,492 missing values
  - `subscription_type`: 4,509 missing values
- Several columns require data type conversion:
  - `user_id`, `session_id`, `app_version` → string
  - `session_start` and `session_end` → datetime
  - `group`,`device_type`,`os`, `language_preference`,`referral_source`,`subscription_type` → categorical
  - `is_premium_user`-> boolean
- Negative values are present in the `total_watch_time_minutes` column and represent invalid observations that will be addressed in the cleaning phase.
- A total of **1,223 fully duplicated rows** were identified and will be removed in the cleaning phase.
- Overall, the dataset structure aligns well with the expected experimental design and is suitable for further cleaning, feature engineering, and analysis.

#### 3. Data Cleaning 
This section prepares the dataset for reliable analysis by resolving logical inconsistencies, correcting invalid values, handling missing data, removing duplicates, and enforcing appropriate data types.

##### **3.1. Logical Consistency Checks**
This section evaluates logical validation rules to ensure internal consistency between related variables. These checks verify that session behavior follows expected business logic and that no structurally impossible records exist.

The following rules are assessed:

- `session_end` must occur after `session_start`
- `clicks` must not exceed `impressions`
- `content_started` must not exceed `clicks`
- `content_completed` must not exceed `content_started`
- `total_watch_time_minutes` must not exceed session duration

In [9]:
# Count sessions where session_end occurs before session_start
invalid_time_order = (df["session_end"] < df["session_start"]).sum()
print(f"Sessions with invalid timestamp order: {invalid_time_order}")

Sessions with invalid timestamp order: 0


In [10]:
# Count sessions where clicks exceed impressions
clicks_exceed_impressions = (df["clicks"] > df["impressions"]).sum()
print(f"Sessions where clicks exceed impressions: {clicks_exceed_impressions}")

Sessions where clicks exceed impressions: 723


In [11]:
# Count sessions where content_started exceeds clicks
started_exceed_clicks = (df["content_started"] > df["clicks"]).sum()
print(f"Sessions where content_started exceeds clicks: {started_exceed_clicks}")

Sessions where content_started exceeds clicks: 0


In [12]:
# Count sessions where content_completed exceeds content_started
completed_exceed_started = (df["content_completed"] > df["content_started"]).sum()
print(f"Sessions where content_completed exceeds content_started: {completed_exceed_started}")

Sessions where content_completed exceeds content_started: 0


In [13]:
# Convert session_start and session_end to datetime
df['session_start'] = pd.to_datetime(df['session_start'])
df['session_end'] = pd.to_datetime(df['session_end'])

# Count sessions where total_watch_time_minutes exceeds calculated session duration
watch_exceeds_duration =  (df["total_watch_time_minutes"] > ((df["session_end"] - df["session_start"]).dt.total_seconds() / 60)).sum()
print(f"Sessions where watch time exceeds session duration: {watch_exceeds_duration}")

Sessions where watch time exceeds session duration: 0


#####  **3.2. Clean Incorrect or Inconsistent Values**

This section resolves the logical inconsistencies identified during validation and standardizes inconsistent categorical representations. The objective is to ensure that all remaining records comply with defined business rules before handling missing values and duplicates.

In [14]:
# Store initial row count
initial_rows = len(df)

# Remove sessions where clicks exceed impressions
df = df[df["clicks"] <= df["impressions"]]

# Calculate remowed rows
removed_click_rows = initial_rows - len(df)
print(f"Rows removed (clicks > impressions): {removed_click_rows}")

# Confirm removal of contradictory rows
remaining_violations = (df["clicks"] > df["impressions"]).sum()
print(f"Remaining sessions where clicks exceed impressions: {remaining_violations}")

Rows removed (clicks > impressions): 723
Remaining sessions where clicks exceed impressions: 0


In [15]:
# Checking the number of rows with negative amount
negative_watch_time = (df["total_watch_time_minutes"] < 0).sum()
print(f"Number of rows with negative watch time values: {negative_watch_time}")

# Store initial row count
before_negative_removal = len(df)

# Remove rows with negative total_watch_time_minutes
df = df[df["total_watch_time_minutes"] >= 0]

# Calculate remowed rows
removed_negative_rows = before_negative_removal - len(df)

# Confirm no negative values remain
remaining_negative_watch_time = (df["total_watch_time_minutes"] < 0).sum()
print(f"Remaining negative watch time values: {remaining_negative_watch_time}")

Number of rows with negative watch time values: 452
Remaining negative watch time values: 0


In [16]:
# Inspect original unique values for potential inconsistencies
cols = [
    "group", "device_type", "os", "app_version",
    "language_preference", "referral_source",
    "subscription_type", "is_premium_user"
]

for col in cols:
    print(f"{col}:", df[col].unique())
    print()

group: ['control' 'TREATMENT' 'treatment' 'TReatent' 'CONTROL' 'coNTroL' 'coNrol'
 'tretment' 'TreaTmeNT' 'cntrol']

device_type: ['tablet' 'smart_tv' 'desktop' 'mobile' 'moblie' 'desktp' nan]

os: ['Android' 'Andriod' 'iOS' 'Windows' 'mac' 'macOS']

app_version: ['5.0.0' "version1'" '6.0.0' '6.1.1' '5.2.1' '5.2.0' nan '5.7.0' '5.5.1'
 '6.2.1' '5.1.1' '5.8.0' '5.8.1' '5.3.0' '5.4.0' '5.9.0' '5.1.0' '5.6.1'
 '6.2.0' '6.1.0' '5.7.1' '5.6.0' '6.3.0' '5.0.1' '5.0.8' '6.0.1' '5.5.0'
 '5.3.1' '5.9.1' '5.4.1']

language_preference: ['Spanish' 'tr-TR' 'dutch' 'DE' 'spanish' 'English' 'fr' nan 'english'
 'EN' 'de' 'Dutch' 'german' 'hindi' 'it' 'hi' 'pt' 'Spnish' 'Englsh']

referral_source: ['push_notification' 'ad_campaign' 'email' 'organic' nan]

subscription_type: ['Basic' 'Premium' 'premium' 'Family' 'Premuim' nan]

is_premium_user: ['False' '0' 'True' 'No' '1' 'yes' 'true']



In [17]:
# Convert group values to lowercase for consistent comparison
df["group"] = df["group"].str.lower()

# Standardize treatment variations
df.loc[df["group"].isin(["treatment", "treatent", "tretment"]), "group"] = "treatment"

# Standardize control variations
df.loc[df["group"].isin(["control", "conrol", "cntrol"]), "group"] = "control"

# Check remaining unique values
df["group"].unique()

array(['control', 'treatment'], dtype=object)

In [18]:
# Correct known misspellings in device_type
df['device_type'] = df['device_type'].replace({'moblie': 'mobile', 'desktp':'desktop'})

# Verify unique values after standardization
df["device_type"].unique()

array(['tablet', 'smart_tv', 'desktop', 'mobile', nan], dtype=object)

In [19]:
# Correct known misspellings in os
df['os'] = df['os'].replace({'Andriod': 'Android', 'mac': 'macOS'})

# Verify unique values after standardization
df["os"].unique()

array(['Android', 'iOS', 'Windows', 'macOS'], dtype=object)

In [20]:
# Correct version1 to unknown in app_version
df['app_version'] = df['app_version'].replace({"version1'": 'Unknown'})

# Verify unique values after standardization
df["app_version"].unique()

array(['5.0.0', 'Unknown', '6.0.0', '6.1.1', '5.2.1', '5.2.0', nan,
       '5.7.0', '5.5.1', '6.2.1', '5.1.1', '5.8.0', '5.8.1', '5.3.0',
       '5.4.0', '5.9.0', '5.1.0', '5.6.1', '6.2.0', '6.1.0', '5.7.1',
       '5.6.0', '6.3.0', '5.0.1', '5.0.8', '6.0.1', '5.5.0', '5.3.1',
       '5.9.1', '5.4.1'], dtype=object)

In [21]:
# Standardize data in language_preference 
df['language_preference'] = df['language_preference'].replace({
    'tr-TR': 'Turkish',
    'DE': 'German',
    'fr': 'French',
    'EN': 'English',
    'de': 'German',
    'it': 'Italian',
    'hi': 'Hindi',
    'pt': 'Portuguese',
    'Spnish': 'Spanish',
    'Englsh': 'English'
    })

# Standardize language_preference to title case (first letter uppercase, rest lowercase)
df["language_preference"] = df["language_preference"].str.title()

# Verify unique values after standardization
df["language_preference"].unique()

array(['Spanish', 'Turkish', 'Dutch', 'German', 'English', 'French', nan,
       'Hindi', 'Italian', 'Portuguese'], dtype=object)

In [22]:
# Correct known misspellings in subscription_type
df['subscription_type'] = df['subscription_type'].replace({'Premuim': 'Premium'})

# Standardize subscription_type to title case (first letter uppercase, rest lowercase)
df["subscription_type"] = df["subscription_type"].str.title()

# Verify unique values after standardization
df["subscription_type"].unique()

array(['Basic', 'Premium', 'Family', nan], dtype=object)

In [23]:
# Standardize boolean-like values to numeric 0/1
df["is_premium_user"] = df["is_premium_user"].replace({
    "False": 0,
    "True": 1,
    "true": 1,
    "false": 0,
    "No": 0,
    "yes": 1,
    "0": 0,
    "1": 1
})

df['is_premium_user'] = df["is_premium_user"].astype("boolean")

# Verify unique values after standardization
df["is_premium_user"].unique()

C:\Users\Tulajdonos\AppData\Local\Temp\ipykernel_25476\202884247.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["is_premium_user"] = df["is_premium_user"].replace({


<BooleanArray>
[False, True]
Length: 2, dtype: boolean

##### **3.3. Handle Missing Values**
This section evaluates the extent and distribution of missing values across the dataset and applies appropriate treatment strategies. The objective is to ensure that missing data does not bias metric calculation or statistical testing.

In [24]:
# Calculate missing value counts and percentages
missing_summary = pd.DataFrame({
    "Missing Count": df.isna().sum(),
    "Missing Percentage": (df.isna().mean() * 100).round(2)
})

# Display only columns with missing values
missing_summary[missing_summary["Missing Count"] > 0]

,Missing Count,Missing Percentage
device_type,2980,2.00
app_version,7223,4.85
language_preference,8046,5.40
referral_source,4453,2.99
subscription_type,4483,3.01


In [25]:
# Define categorical metadata columns with missing values
categorical_missing_cols = [
    "device_type",
    "app_version",
    "language_preference",
    "referral_source",
    "subscription_type"
]

# Fill missing values with "Unknown"
df[categorical_missing_cols] = df[categorical_missing_cols].fillna("Unknown")

# Confirm no missing values remain in these columns
df[categorical_missing_cols].isna().sum()

device_type            0
app_version            0
language_preference    0
referral_source        0
subscription_type      0
dtype: int64

##### **3.4. Remove Duplicated Rows**
This section identifies and removes fully duplicated records to ensure that each session observation is unique. Duplicate removal prevents distortion of engagement metrics and preserves the integrity of statistical testing.

In [26]:
# Calculate duplicate count
duplicate_count = df.duplicated().sum()

# Calculate percentage of duplicates
duplicate_percentage = (duplicate_count / len(df)) * 100

print(f"Duplicate rows: {duplicate_count}")
print(f"Duplicate percentage: {duplicate_percentage:.2f}%")

Duplicate rows: 1235
Duplicate percentage: 0.83%


In [27]:
# Remove fully duplicated rows
df = df.drop_duplicates()

# Confirm no duplicate rows remain
remaining_duplicates = df.duplicated().sum()
print(f"Remaining duplicate rows: {remaining_duplicates}")

Remaining duplicate rows: 0


In [28]:
# Display dataset dimensions
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Number of rows: 147669
Number of columns: 17


##### **3.5. Fix Incorrect Column Data Types**
This section finalizes the dataset schema by enforcing appropriate data types for identifiers, timestamps, categorical variables, boolean fields, and numeric columns. Ensuring correct data types improves memory efficiency, analytical reliability, and downstream metric computation.

In [29]:
# Convert identifier columns to string
df["user_id"] = df["user_id"].astype("string")
df["session_id"] = df["session_id"].astype("string")
df["app_version"] = df["app_version"].astype("string")

# Convert timestamp columns to datetime
df['session_start'] = pd.to_datetime(df['session_start'])
df['session_end'] = pd.to_datetime(df['session_end'])

# Define categorical columns
categorical_columns = [
    "group",
    "device_type",
    "os",
    "language_preference",
    "referral_source",
    "subscription_type"
]

# Convert to category dtype
df[categorical_columns] = df[categorical_columns].astype("category")

# Convert is_premium_user to boolean dtype
df["is_premium_user"] = df["is_premium_user"].astype("boolean")

# Display final data types
df.dtypes

user_id                     string[python]
group                             category
session_id                  string[python]
session_start               datetime64[ns]
session_end                 datetime64[ns]
impressions                          int64
clicks                               int64
content_started                      int64
content_completed                    int64
total_watch_time_minutes           float64
device_type                       category
os                                category
app_version                 string[python]
language_preference               category
referral_source                   category
subscription_type                 category
is_premium_user                    boolean
dtype: object

##### **Summary of Data Cleaning:**

- The dataset was **reduced** from 150,079 rows **to 147,669 rows**, following the removal of structurally invalid and duplicated observations.
- A total of **723 rows** where `clicks` exceeded `impressions` were removed to preserve the validity of engagement metrics.
- **452 rows** containing negative values in `total_watch_time_minutes` were eliminated as invalid behavioral observations.
- A total of **1,235 duplicated session records** were identified and removed to ensure session-level uniqueness.
- Categorical inconsistencies across `group`, `device_type`, `os`, `app_version`, `language_preference`, `subscription_type`, and `is_premium_user` were standardized to ensure consistent representation.
- Missing values were present only in secondary metadata fields and were imputed with `"Unknown"` to preserve dataset size while maintaining transparency.
- All columns were assigned appropriate data types (string, datetime, category, boolean, numeric) to enforce a consistent analytical schema.

Overall, the cleaned dataset is structurally consistent, free of logical contradictions, missing values, and duplicates, and ready for exploratory analysis and statistical testing.

#### 4. Post-Cleaning Data Inspection

This section validates the final structure of the cleaned dataset by confirming row count changes, the absence of missing values and duplicates, and correct data type enforcement.

In [30]:
# Preview the first five rows
df.head()

,user_id,group,session_id,session_start,session_end,impressions,clicks,content_started,content_completed,total_watch_time_minutes,device_type,os,app_version,language_preference,referral_source,subscription_type,is_premium_user
0,8116,control,48680,2026-01-11 12:07:36.131448076,2026-01-11 12:52:36.131448076,21,6,4,1,22.08,tablet,Android,5.0.0,Spanish,push_notification,Basic,False
1,12109,control,72536,2026-01-14 10:02:48.687211687,2026-01-14 11:11:48.687211687,24,8,8,2,37.17,smart_tv,Android,Unknown,Turkish,push_notification,Basic,False
2,15861,control,94927,2026-01-14 10:28:05.164624346,2026-01-14 11:59:05.164624346,21,3,3,1,34.53,desktop,iOS,6.0.0,Dutch,ad_campaign,Premium,False
3,21199,treatment,127477,2026-01-03 20:33:22.917005566,2026-01-03 22:06:22.917005566,24,6,6,5,66.34,desktop,Android,6.1.1,German,email,Premium,True
4,2324,control,13847,2026-01-18 05:00:07.290494422,2026-01-18 06:52:07.290494422,15,3,2,1,79.78,mobile,Windows,5.2.1,Spanish,push_notification,Premium,False


In [31]:
# Display dataset dimensions
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Number of rows: 147669
Number of columns: 17


In [32]:
# Display column names, data types, and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147669 entries, 0 to 150078
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   user_id                   147669 non-null  string        
 1   group                     147669 non-null  category      
 2   session_id                147669 non-null  string        
 3   session_start             147669 non-null  datetime64[ns]
 4   session_end               147669 non-null  datetime64[ns]
 5   impressions               147669 non-null  int64         
 6   clicks                    147669 non-null  int64         
 7   content_started           147669 non-null  int64         
 8   content_completed         147669 non-null  int64         
 9   total_watch_time_minutes  147669 non-null  float64       
 10  device_type               147669 non-null  category      
 11  os                        147669 non-null  category      
 12  app_ver

In [33]:
# Show descriptive statistics for the numerical columns
df.describe()

,session_start,session_end,impressions,clicks,content_started,content_completed,total_watch_time_minutes
count,147669,147669,147669.000000,147669.000000,147669.000000,147669.000000,147669.000000
mean,2026-01-11 11:07:42.492608,2026-01-11 12:12:36.406022400,19.984059,3.101314,2.792015,1.662096,39.277047
min,2026-01-01 00:00:15.905562108,2026-01-01 00:12:20.646265242,0.000000,0.000000,0.000000,0.000000,0.390000
25%,2026-01-06 04:45:21.582897408,2026-01-06 05:50:12.909004288,17.000000,2.000000,2.000000,1.000000,19.240000
50%,2026-01-11 11:18:28.737827584,2026-01-11 12:24:45.190423808,20.000000,3.000000,3.000000,1.000000,35.360000
75%,2026-01-16 16:59:37.232501760,2026-01-16 18:08:48.781817600,23.000000,4.000000,4.000000,2.000000,55.800000
max,2026-01-21 23:44:07.878168519,2026-01-21 23:59:57.756882343,42.000000,14.000000,14.000000,10.000000,119.040000
std,NaN,NaN,5.023808,1.801089,1.703575,1.303175,24.142438


In [34]:
# Count missing values per column
df.isnull().sum()

user_id                     0
group                       0
session_id                  0
session_start               0
session_end                 0
impressions                 0
clicks                      0
content_started             0
content_completed           0
total_watch_time_minutes    0
device_type                 0
os                          0
app_version                 0
language_preference         0
referral_source             0
subscription_type           0
is_premium_user             0
dtype: int64

In [35]:
# Check for fully duplicated rows
print(f"Number of fully duplicated rows:{df.duplicated().sum()}")

Number of fully duplicated rows:0


In [36]:
# Verify unique values 
for col in [
    "group",
    "device_type",
    "os",
    "app_version",
    "language_preference",
    "referral_source",
    "subscription_type",
    "is_premium_user"
]:    
    print(f"\n{col}")
    print(df[col].value_counts(dropna=False))


group
group
control      73865
treatment    73804
Name: count, dtype: int64

device_type
device_type
tablet      36316
smart_tv    36223
desktop     36144
mobile      36029
Unknown      2957
Name: count, dtype: int64

os
os
Android    40850
macOS      40439
Windows    33241
iOS        33139
Name: count, dtype: int64

app_version
app_version
Unknown    11587
6.2.0       4999
5.5.1       4961
6.0.1       4955
5.7.0       4953
6.0.0       4905
5.1.1       4899
5.3.1       4896
5.5.0       4894
5.1.0       4883
5.4.0       4879
6.3.0       4870
6.1.0       4869
5.0.1       4863
5.2.1       4857
5.8.1       4855
5.6.1       4855
5.9.1       4849
5.0.0       4847
6.2.1       4844
5.7.1       4837
5.3.0       4833
5.2.0       4818
6.1.1       4816
5.9.0       4810
5.8.0       4785
5.0.8       4768
5.6.0       4745
5.4.1       4737
Name: count, dtype: Int64

language_preference
language_preference
English       41362
German        30038
Dutch         27837
Spanish       27733
Unknown        7

In [37]:
# Count how many unique groups each user_id appears in
user_group_counts = df.groupby("user_id")["group"].nunique()

# Check whether all users belong to exactly one group
if (user_group_counts == 1).all():
    print("Experiment Integrity Check Passed: All users are assigned to exactly one group (no cross-over contamination detected).")
else:
    print("Experiment Integrity Check Failed: Some users appear in multiple groups.")

Experiment Integrity Check Passed: All users are assigned to exactly one group (no cross-over contamination detected).


##### **Post-Cleaning Validation Summary**

- No remaining logical inconsistencies detected
- No duplicate records present
- No unintended missing values in critical fields
- Each user is assigned to exactly one experimental group

The dataset is now validated and ready for exploratory analysis and metric construction.

#### 5. Save the Cleaned Dataset


In [38]:
# Define output path for the cleaned dataset
output_path = "../data/processed/streaming_ab_test_sessions_cleaned.csv"

# Save the cleaned DataFrame to the processed data directory (exclude row index)
df.to_csv(output_path, index=False)

# Confirm successful export
print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: ../data/processed/streaming_ab_test_sessions_cleaned.csv


The cleaned dataset will be used as the input for exploratory analysis and hypothesis testing in subsequent notebooks.